# Volume 4: Wave Equation
    Benj McMullin
    Math 437
    11/30/2023

In [20]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation as ani
from scipy.linalg import toeplitz
import scipy.optimize
from scipy.optimize import fsolve

In [2]:
def wave_solver(x, t, s, f, g):
    '''
    This Function solves the wave equation u_tt = u_xx for x in [0, 1] and t in [0, T]
    with boundary conditions u(0, t) = u(1, t) = 0 and initial conditions u(x, 0) = f(x) and u_t(x, 0) = g(x)
    using the finite difference method.
    The function returns a matrix U of size (len(t), len(x)) where U[i][j] is the value of u(x[j], t[i]).
    '''

    # x: array of x values
    dx = x[1] - x[0]
    dt = t[1] - t[0]
    lam = s * dt / dx
    N = len(x)

    # create A
    diag_1 = np.ones(N) * 2 * (1 - lam ** 2)
    diag_2 = np.ones(N - 1) * lam ** 2
    A = np.diag(diag_1) + np.diag(diag_2, k = 1) + np.diag(diag_2, k = -1)

    # create U
    U0 = f(x)
    U1 = U0 + dt * g(x) + 0.5 * lam ** 2 * (np.roll(U0, -1) - 2 * U0 + np.roll(U0, 1))
    U = np.zeros((len(t), N))
    U[0] = U0
    U[1] = U1

    # solve for U
    for i in range(2, len(t)):
        a = A @ U[i - 1]
        b = U[i - 2]
        U[i][1: -1] = a[1: -1] - b[1: -1]
        U[i][0] = 0
        U[i][-1] = 0
    
    return U

In [12]:
def animate(x, u, title, ylim=None):
    '''
    This function animates the solution to the wave equation.
    '''

    # create figure
    figure = plt.figure()
    plt.ioff()
    ax1 = figure.add_subplot(1, 1, 1)

    # set up axes
    ax1.set_xlim([x[0], x[-1]])
    if ylim:
        ax1.set_ylim(ylim)
    else:
        ax1.set_ylim((min(min(r) for r in u) * 1.25, max(max(r) for r in u) * 1.25))
    ax1.set_xlabel('x')
    ax1.set_ylabel('u(x,t)')
    ax1.set_title(title)

    # create line
    line, = ax1.plot([], [], lw = 2)
    def update(i):
        line.set_data(x, u[i])
        return line,

    plt.ioff()
    ani = animation.FuncAnimation(figure, update, frames = range(u.shape[0]), interval = 25, repeat = True)
    ani.save(f'{title}.mp4')

# Problem 1
Numerically approximate the solution to the following BVP

$$
u_{tt} = u_{xx} \\
u(0,t) = u(1,t) = 0, \\
u(x,0) = \sin(2\pi x), \\
u_t(x,0) = 0. 
$$

Numerically approximate the solution $u(x,t)$ for $t\in[0,0.5]$. Use $J=50$ subintervals in the $x$ dimension and $M=50$ subintervals in the $t$ dimension. Animate the results. Compare you results with the analytic solution $u(x,t) = \sin(2\pi x)\cos(2\pi t).$ This function is know as a standing wave.

In [13]:
# Initalize variables x and t which are arrays of x and t values
x = np.linspace(0, 1, 51)
t = np.linspace(0, 0.5, 51)

# Create functions f, g, and s
f = lambda x: np.sin(2 * np.pi * x)
g = lambda x: np.zeros_like(x)
s = 1

# Solve the wave equation and animate the solution
u = wave_solver(x, t, s, f, g)
animate(x, u, 'Problem 1')

# Problem 2

Consider the boundary value problem 

$$
u_{tt} = u_{xx}, \\
u(0,t) = u(1,t) = 0, \\
u(x,0) = 0.2e^{-m^2(x-1/2)^2} \\
y_t(x,0) = 0.4m^2(x-\frac{1}{2})e^{-m^2(x-1/2)^2}.
$$

The solution of this problem is ta Gaussian pulse. It travels to the right at a constant speed. This solution models, for example, a wave pulse in a stretched string. Note that the fixed boundart conditions reflect the pulse back when it meets the boundary.

Numerically approximate the solution $u(x,t)$ for $t\in[0,1]$. Set $m=20$. Use 200 subintervals in space and 220 in time, and animate your results. Then use 200 subintervals in space and 180 in time, and animate your results. Note that the stability condition is not satisfied for the second mesh.

In [14]:
# Initalize variables x and t which are arrays of x and t values
x = np.linspace(0, 1, 201)
t = np.linspace(0, 1, 221)
m = 20

# Create functions f, g, and s
f = lambda x: 0.2 * np.exp(-m ** 2 * (x - 0.5) ** 2)
g = lambda x: 0.4 * m ** 2 * (x - 0.5) * np.exp(-m ** 2 * (x - 0.5) ** 2)

# Solve the wave equation and animate the solution
u = wave_solver(x, t, s, f, g)
animate(x, u, 'Problem 2a', ylim = (-0.5, 0.5))

In [15]:
# Initalize variables x and t which are arrays of x and t values
x = np.linspace(0, 1, 201)
t = np.linspace(0, 1, 181)
m = 20

# Create functions f, g, and s
f = lambda x: 0.2 * np.exp(-m ** 2 * (x - 0.5) ** 2)
g = lambda x: 0.4 * m ** 2 * (x - 0.5) * np.exp(-m ** 2 * (x - 0.5) ** 2)

# Solve the wave equation and animate the solution
u = wave_solver(x, t, s, f, g)
animate(x, u, 'Problem 2b', ylim = (-1, 1))

# Problem 3

Consider the initial boundary value problem
$$
	u_{tt} = u_{xx}, \\
	u(0,t) = u(1,t) = 0, \\
	u(x,0) = 0.2e^{-m^2(x-1/2)^2}\\
	u_t(x,0) = 0.
$$
The initial condition separates into two smaller, slower-moving pulses, one travelling to the right and the other to the left.
This solution models, for example, a plucked guitar string

Numerically approximate the solution $u(x,t)$ for $t \in \left[0,2\right]$.
Set $m=20$.
Use 200 subintervals in space and 440 in time, and animate your results.
It is rather easy to see that the solution to this problem is the sum of two travelling waves, one travelling to the left and the other to the right, as described earlier.

In [16]:
# Initalize variables x and t which are arrays of x and t values
x = np.linspace(0, 1, 201)
t = np.linspace(0, 2, 441)
m = 20

# Create functions f, g, and s
f = lambda x: 0.2 * np.exp(-m ** 2 * (x - 0.5) ** 2)
g = lambda x: np.zeros_like(x)

u = wave_solver(x, t, s, f, g)
animate(x, u, 'Problem 3', ylim = (-0.5, 0.5))

# Problem 4

Consider the initial boundary value problem

$$
	u_{tt} = u_{xx}, \\
	u(0,t) = u(1,t) = 0, \\
	u(x,0) = \begin{cases} 1/3 & \text{if } 5/11 < x < 6/11,\\
	0 & \text{otherwise}
	\end{cases}\\
	u_t(x,0) = 0.
$$

Numerically approximate the solution $u(x,t)$ for $t \in \left[0, 2\right]$.
Use 200 subintervals in space and 440 in time, and animate your results.
Even though the method is second order and stable for this discretization, since the initial condition is discontinuous there are large dispersive errors.

In [17]:
# Initalize variables x and t which are arrays of x and t values
x = np.linspace(0, 1, 201)
t = np.linspace(0, 1, 441)

# Create functions f, g, and s
f = lambda x: np.where((x > 5 / 11) & (x < 6 / 11), 1 / 3, 0)
g = lambda x: np.zeros_like(x)

# Solve the wave equation and animate the solution
u = wave_solver(x, t, s, f, g)
animate(x, u, 'Problem 4', ylim = (-0.5, 0.5))

# Problem 5


Numerically solve the initial value problem

$$
	u_t -su_x + uu_x = u_{xx}, \quad x \in (-\infty,\infty),\\
	u(x,0) = v(x),
$$

for $t \in [0,1]$.
Let the perturbation $v(x)$ be given by

$$
v(x) = 3.5(\sin{(3x)} + 1)\frac{1}{\sqrt{2\pi}} \exp{(-x^2/2)}
$$

And let the initial condition be $u(x, 0) = \hat{u}(x) + v(x)$
Approximate the $x$ domain,$(-\infty, \infty)$, numerically by the finite interval $[-20,20]$, and fix $u(-20) = u_-$, $u(20) = u_+$. Let $u_- = 5$, $u_+ = 1$.
Use 150 intervals in space and 350 steps in time.
Animate your results.
You should see the solution converge to a translate of the travelling wave $\hat{u}$.

Hint: This difference scheme is no longer a linear equation.
We have a nonlinear equation in $U^{n+1}$.
We can still solve this function using Newton's method or some other similar solver.
In this case, use `scipy.optimize.fsolve`.

In [26]:
def solve_burgers(x, t, f, u_minus, u_plus):
    """
    This function solves the Burgers' equation u_t + uu_x = 0 for x in [0, 1] and t in [0, T]
    with boundary conditions u(0, t) = u_minus and u(1, t) = u_plus and initial condition u(x, 0) = f(x)
    using the finite difference method.
    The function returns a matrix U of size (len(t), len(x)) where U[i][j] is the value of u(x[j], t[i]).
    """
    # Create variables and constants
    dx = x[1] - x[0]
    dt = t[1] - t[0]
    k1 = dt / 4 / dx
    k2 = dt / 2 / dx ** 2
    s = (u_minus + u_plus) / 2

    # create U
    U = np.zeros((len(x), len(t)))
    U[:, 0] = f(x)
    U[0, 0], U[0, -1] = u_minus, u_plus

    # solve for U
    def crank(u1, u0, s, k1, k2):

        col = u1.shape[-1]
        result = [u1[i] - u0[i] - k1 * ((s - u1[i]) * (u1[i + 1] - u1[i - 1]) + (s - u0[i]) * (u0[i + 1] - u0[i - 1])) - \
                    k2 * ((u1[i + 1] - 2 * u1[i] + u1[i - 1]) + (u0[i + 1] - 2 * u0[i] + u0[i - 1])) for i in range(1, col - 1)]
        return [u1[0] - u0[0]] + result + [u1[-1] - u0[-1]]
    
    # solve for U    
    for i in range(1, U.shape[0] - 1):
        U[:, i] = fsolve(crank, U[:, 0], args = (U[:, i - 1], s, k1, k2))
    
    return U

In [27]:
# Initalize variables x and t which are arrays of x and t values
x = np.linspace(-20, 20, 151)
t = np.linspace(0, 1, 351)

# Create functions f, g, and s
nu = 1
Δ = 0
minus_u = 5
plus_u = 1
a = (minus_u - plus_u) / 2

# Solve the wave equation and animate the solution
u_hat = lambda x: s - a * np.tanh(a * x / (2 * nu) + Δ)
v = lambda x: 3.5 * (np.sin(3 * x) + 1) * np.exp(-x ** 2 / 2) / np.sqrt(2 * np.pi)
f = lambda x: u_hat(x) + v(x)

u = solve_burgers(x, t, f, minus_u, plus_u)

In [28]:
# In this cell we create the animation for problem 5

# Create figure
figure = plt.figure()
wave, = plt.plot([], [], lw = 2)
plt.xlim([x[0], x[-1]])
plt.ylim((-1, 6))
plt.xlabel('x')
plt.ylabel('u(x,t)')
plt.title('Problem 5')

# Create animation using the update function
def update(i):
    wave.set_data(x, u[:, i])
    return wave,

# Save animation
ani = animation.FuncAnimation(figure, update, frames = range(len(u)), interval = 25, repeat = True)
ani.save(f'problem5.mp4')